In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
import datetime
from matplotlib import pyplot as plt

%matplotlib inline
%run ./data_loading.ipynb

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5, stride=1, padding=0)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(in_features=10*10*12, out_features=300)
        self.fc2 = nn.Linear(in_features=300, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=9)
        
    def forward(self, X):
        X = self.pool(self.relu(self.conv1(X)))
        X = self.pool(self.relu(self.conv2(X)))
        X = X.view(-1, 10*10*12)
        X = self.relu(self.fc1(X))
        X = self.relu(self.fc2(X))
        X = self.fc3(X)
        return X

In [3]:
net = Net().float()
if torch.cuda.is_available():
    net = net.cuda()

In [4]:
loss_f = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [5]:
training_set = ClassroomDataset()

In [6]:
batch_size = 5
training_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)

In [17]:
for epoch in range(20):
    running_loss = 0.0
    for i, data in tqdm(enumerate(training_loader, 0)):
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_f(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('epoch:', epoch, ', loss:', running_loss * batch_size / training_loader.__len__())

640it [00:04, 135.09it/s]


epoch: 0 , loss: 0.033343842556586845


640it [00:05, 120.64it/s]


epoch: 1 , loss: 0.027916130855770493


640it [00:03, 170.98it/s]


...

epoch: 17 , loss: 0.005596013076302198


640it [00:04, 114.13it/s]


epoch: 18 , loss: 0.0043293934443027204


640it [00:05, 107.12it/s]


epoch: 19 , loss: 0.0042523507282936635


In [18]:
p = "weights/" + str(datetime.datetime.now()).split(".")[0].replace(':','_') + ".pth"
torch.save(net.state_dict(), p)
print(p)

weights/2019-11-27 16_11_41.pth


# Testing

In [19]:
test_set = ClassroomDataset(test_set=True)

In [20]:
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [21]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('test accuracy: %d %%' % (
    100 * correct / total))

test accuracy: 91 %


In [22]:
correct = 0
total = 0
with torch.no_grad():
    for data in training_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('train accuracy: %d %%' % (
    100 * correct / total))

train accuracy: 99 %
